# HW2 - Bias in Data and Prediction - DSCI 531 - Spring 2025

### Please complete the code or analysis under "TODO". 80pts in total. You should run every cell and keep all the outputs before submitting. Failing to include your outputs will result in zero points.
### Please keep academic integrity in mind. Plagiarism will be taken seriously.

#### Name: Liujia Yu
#### USC ID: 4764432021

In [1]:
import numpy as np
import pandas as pd

## 1. Implement Utility Functions

### 1.1 Fairness Metrics

In [2]:
# You are NOT allowed to use off-the-shelf fairness packages like ai360

def stat_parity(preds, sens):
    '''
    :preds: numpy array of the model predictions. Consisting of 0s and 1s
    :sens: numpy array of the sensitive features. Consisting of 0s and 1s
    :return: the statistical parity. no need to take the absolute value
    '''
    
    # TODO. 10pts
    p_0 = np.mean(preds[sens == 0])
    p_1 = np.mean(preds[sens == 1])
    return p_0 - p_1


def eq_oppo(preds, sens, labels):
    '''
    :preds: numpy array of the model predictions. Consisting of 0s and 1s
    :sens: numpy array of the sensitive features. Consisting of 0s and 1s
    :labels: numpy array of the ground truth labels of the outcome. Consisting of 0s and 1s
    :return: the equalized odds. no need to take the absolute value
    '''
    
    # TODO. 10pts
    # TPR for group of sens is 0
    true_posi_mask = (sens == 0) & (labels == 1)
    all_posi_cnt = np.sum(labels[sens == 0])
    tpr_0 = ( np.sum(preds[true_posi_mask]) / all_posi_cnt ) \
                if all_posi_cnt > 0 else 0 # make sure the divider is not 0

    # TPR for group of sens is 1
    true_posi_mask = (sens == 1) & (labels == 1)
    all_posi_cnt = np.sum(labels[sens == 1])
    tpr_1 = ( np.sum(preds[true_posi_mask]) / all_posi_cnt ) \
                if all_posi_cnt > 0 else 0
    
    tpr_diff = tpr_0 - tpr_1

    # FPR for group of sens is 0
    false_posi_mask = (sens == 0) & (labels == 0)
    all_nega_cnt = np.sum(1 - labels[sens == 0])
    fpr_0 = ( np.sum(preds[false_posi_mask]) / all_nega_cnt ) \
                if all_nega_cnt > 0 else 0

    # FPR for group of sens is 1
    false_posi_mask = (sens == 1) & (labels == 0)
    all_nega_cnt = np.sum(1 - labels[sens == 1])
    fpr_1 = ( np.sum(preds[false_posi_mask]) / all_nega_cnt ) \
                if all_nega_cnt > 0 else 0
    
    fpr_diff = fpr_0 - fpr_1

    return tpr_diff + fpr_diff

In [3]:
# Test your implemented fairness metrics using the code below
# Don't change the code in this cell

# test case 1
preds = np.array([1, 0, 1, 0, 0, 1, 0, 0, 0, 1])
sens = np.array([1, 1, 0, 1, 1, 1, 0, 1, 1, 1])
labels = np.array([0, 1, 0, 1, 0, 1, 1, 1, 0, 1])
print(eq_oppo(preds, sens, labels), stat_parity(preds, sens))

# test case 2
preds = np.array([1, 1, 0, 1, 0, 1, 0, 0, 1, 1])
sens = np.array([1, 0, 0, 0, 0, 0, 0, 0, 0, 1])
labels = np.array([0, 1, 0, 1, 0, 1, 1, 0, 0, 0])
print(eq_oppo(preds, sens, labels), stat_parity(preds, sens))


# test case 3
preds = np.array([1, 0, 0, 0, 0, 0, 0, 0, 0, 1])
sens = np.array([1, 0, 0, 0, 0, 0, 0, 0, 0, 1])
labels = np.array([0, 1, 0, 1, 0, 1, 1, 0, 0, 0])
print(eq_oppo(preds, sens, labels), stat_parity(preds, sens))

0.2666666666666667 0.125
0.0 -0.5
-1.0 -1.0


### 1.2 Preprocessing DataFrame

In [4]:
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
# def min_max_normalize(df, numerical_cols):
#     for col in numerical_cols:
#         min_val = df[col].min()
#         max_val = df[col].max()
#         df[col] = (df[col] - min_val) / (max_val - min_val)
#     return df

def process_dfs(df_train_x, df_test_x, categ_cols):
    '''
    Pre-process the features of the training set and the test set, not including the outcome column.
    Convert categorical features (nominal & ordinal features) to one-hot encodings.
    Normalize the numerical features into [0, 1].
    We process training set and the test set together in order to make sure that 
    the encodings are consistent between them.
    For example, if one class is encoded as 001 and another class is encoded as 010 in the training set,
    you should follow this mapping for the test set too.
    
    :df_train: the dataframe of the training data
    :df_test: the dataframe of the test data
    :categ_cols: the column names of the categorical features. the rest features are treated as numerical ones.
    :return: the processed training data and test data, both should be numpy arrays, instead of DataFrames
    '''
    
    # TODO. 10pts
    # Create copies to avoid modifying original data
    df_train = df_train_x.copy()
    df_test = df_test_x.copy()
    
    # Identify numerical columns (those not in categ_cols)
    numeric_cols = [col for col in df_train.columns if col not in categ_cols]
    
    # Handle categorical features
    for col in categ_cols:
        # Get all unique values from both train and test sets
        unique_values = pd.concat([df_train[col], df_test[col]]).unique()
        
        # Create dummy variables for both sets 
        train_dummies = pd.get_dummies(df_train[col], prefix=col, dtype = int)  # sets dtype to int
        test_dummies = pd.get_dummies(df_test[col], prefix=col, dtype = int)
        
        # Ensure both sets have the same dummy columns
        # because when dataset is small, some values might be missing therefore can't create certain cols using OneHotEncoding
        missing_cols = set(train_dummies.columns) - set(test_dummies.columns)
        for c in missing_cols:
            test_dummies[c] = 0
        missing_cols = set(test_dummies.columns) - set(train_dummies.columns)
        for c in missing_cols:
            train_dummies[c] = 0
            
        # Sort columns to ensure same order
        train_dummies = train_dummies.reindex(sorted(train_dummies.columns), axis=1)
        test_dummies = test_dummies.reindex(sorted(test_dummies.columns), axis=1)
        
        # Drop original column and add dummy columns
        df_train = df_train.drop(col, axis=1)
        df_test = df_test.drop(col, axis=1)
        df_train = pd.concat([df_train, train_dummies], axis=1)
        df_test = pd.concat([df_test, test_dummies], axis=1)
    
    # Handle numerical features
    if numeric_cols:
        scaler = MinMaxScaler()
        df_train[numeric_cols] = scaler.fit_transform(df_train[numeric_cols])
        df_test[numeric_cols] = scaler.transform(df_test[numeric_cols])
    
    # Convert to numpy arrays
    # print(df_test.columns)    # Note: for test cases the cols are: ['height', 'size_big', 'size_medium', 'size_small', 'color_blue', 'color_red', 'color_yellow']
    train_x = df_train.values
    test_x = df_test.values
    
    return train_x, test_x

In [5]:
# Test your implemented data preprocessing function
# DO NOT change the code in this cell

df_train_x = pd.DataFrame([
    [ 'big', 10, 'blue',],
    [ 'big', 12, 'red',],
    ['medium', 5, 'blue'],
    ['small', 7, 'yellow']
], columns=['size', 'height', 'color'])

df_test_x = pd.DataFrame([
    [ 'big', 16, 'red',],
    ['small', 9, 'blue']
], columns=['size', 'height', 'color'])

train_data_x, test_data_x = process_dfs(df_train_x, df_test_x, categ_cols=['size', 'color'])
print(train_data_x)
print()
print(test_data_x)

[[0.71428571 1.         0.         0.         1.         0.
  0.        ]
 [1.         1.         0.         0.         0.         1.
  0.        ]
 [0.         0.         1.         0.         1.         0.
  0.        ]
 [0.28571429 0.         0.         1.         0.         0.
  1.        ]]

[[1.57142857 1.         0.         0.         0.         1.
  0.        ]
 [0.57142857 0.         0.         1.         1.         0.
  0.        ]]


## 2. Load Data

In [6]:
df_train_adult = pd.read_csv('adult-train.csv', sep=', ', engine='python')
df_test_adult = pd.read_csv('adult-test.csv', sep=', ', engine='python')
df_train_adult['sex'] = df_train_adult['sex'].map({'Male': 0, 'Female': 1})
df_test_adult['sex'] = df_test_adult['sex'].map({'Male': 0, 'Female': 1})
df_train_adult['income'] = df_train_adult['income'].map({'<=50K': 0, '>50K': 1})
df_test_adult['income'] = df_test_adult['income'].map({'<=50K': 0, '>50K': 1})


df_train_german = pd.read_csv('german-train.csv')
df_test_german = pd.read_csv('german-test.csv')
df_train_german['age'] = df_train_german['age'].apply(lambda x: 1 if x >= 33 else 0)
df_test_german['age'] = df_test_german['age'].apply(lambda x: 1 if x>=33 else 0)
df_train_german['credit_status'] = df_train_german['credit_status'].map({2:0, 1:1})
df_test_german['credit_status'] = df_test_german['credit_status'].map({2:0, 1:1})

In [7]:
df_train_adult.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,0,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,0,0,0,13,United-States,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,0,0,0,40,United-States,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,0,0,0,40,United-States,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,1,0,0,40,Cuba,0


In [8]:
df_train_german.head()

,checking_account,duration,credit_history,purpose,credit_amount,savings_account,present_employment_since,installment_rate,personal_status_sex,other_debtors,...,property,age,other_installment_plans,housing,num_credits,job,num_people_liable,telephone,foreign_worker,credit_status
0,A14,21,A32,A41,5248,A65,A73,1,A93,A101,...,A123,0,A143,A152,1,A173,1,A191,A201,1
1,A11,24,A32,A43,1987,A61,A73,2,A93,A101,...,A121,0,A143,A151,1,A172,2,A191,A201,0
2,A14,36,A32,A49,5742,A62,A74,2,A93,A101,...,A123,0,A143,A152,2,A173,1,A192,A201,1
3,A14,36,A32,A49,7409,A65,A75,3,A93,A101,...,A122,1,A143,A152,2,A173,1,A191,A201,1
4,A14,6,A34,A42,1221,A65,A73,1,A94,A101,...,A122,0,A143,A152,2,A173,1,A191,A201,1


## 3. Explore fairness in data

### 3.1 statical analysis on protected feature and outcome

In [9]:
# Adult
# calculate the mean income of two protected groups. only use the training data df_train_adult. 
# TODO. 2pts. The starter code below just indicate what you need to output in your code.
mean_income1_adult = np.mean(df_train_adult['income'][df_train_adult['sex'] == 0])
mean_income2_adult = np.mean(df_train_adult['income'][df_train_adult['sex'] == 1])

print(mean_income1_adult, mean_income2_adult)


# German
# calculate the mean credit status of two protected groups. only use the training data df_train_german. 
# TODO. 2pts. The starter code below just indicate what you need to output in your code.
mean_credit1_german = np.mean(df_train_german['credit_status'][df_train_german['age'] == 0])
mean_credit2_german = np.mean(df_train_german['credit_status'][df_train_german['age'] == 1])

print(mean_credit1_german, mean_credit2_german)

0.3138370951913641 0.11367818442036394
0.6636363636363637 0.7594594594594595


In [10]:
# t-test between outcome of two protected groups. only use the training data df_train_adult/german.

from scipy.stats import ttest_ind
# import math

# def manual_t_test(group1, group2):
#     mean1, mean2 = sum(group1) / len(group1), sum(group2) / len(group2)
#     var1 = sum((x - mean1) ** 2 for x in group1) / (len(group1) - 1)
#     var2 = sum((x - mean2) ** 2 for x in group2) / (len(group2) - 1)
    
#     se = math.sqrt(var1 / len(group1) + var2 / len(group2))
#     t_stat = (mean1 - mean2) / se
    
#     return abs(t_stat)
# Notes: This function can't return p-values because we have to search for the distribution table

# Adult
# TODO. 3pts. The starter code below just indicate what you need to output in your code.
male_income = df_train_adult[df_train_adult['sex'] == 0]['income']
female_income = df_train_adult[df_train_adult['sex'] == 1]['income']
p_value_adult = ttest_ind(male_income, female_income, equal_var=False).pvalue
# p_value_adult = manual_t_test(male_income, female_income)

# german
# TODO. 3pts. The starter code below just indicate what you need to output in your code.
young_credit = df_train_german[df_train_german['age'] == 0]['credit_status']
older_credit = df_train_german[df_train_german['age'] == 1]['credit_status']
p_value_german = ttest_ind(young_credit, older_credit, equal_var=False).pvalue
# p_value_german = manual_t_test(young_credit, older_credit)

print(p_value_adult, p_value_german)

0.0 0.0053039008376585695


### From the p_values, are the results significant for Adult and German? How do you explain them?
### <span style="color:red">Please type your response here.</span> 3pts

For the adult dataset, the p-value is almost 0. This suggests a very strong statistical difference between the two sex groups regarding income.It means gender significantly impacts income classification in this dataset, indicating potential bias.

For the german dataset, the p-value is 0.0053, which is still much lower than 0.05. It means there exists a statistically significant difference in credit status for different age groups. Age has a non-negligible effect on credit status. Bias might exist in how credit is assigned.

**References:**
1. [T-Test: What It Is With Multiple Formulas and When To Use Them](https://www.investopedia.com/terms/t/t-test.asp#:~:text=A%20t%2Dtest%20is%20an%20inferential%20statistic%20used%20to%20determine,flipping%20a%20coin%20100%20times.)

### 3.2 Explore Fairness in Prediction

In [11]:
# Prepare data
# Dont't change code in this cell

'''
:train_x: the features in the training set (including the sensitive features), shape: N_train x d
:train_y: the outcome in the training set, shape: N_train
:test_x: the features in the test set (including the sensitive features), shape: N_test x d
:test_y: the outcome in the test set, shape: N_test
:test_sens: the sensitive/protected feature in the test set, shape: N_test
All of them are processed numpy arrays that are ready for algorithms.
'''


# adult
# the outcome (income) is the last column
df_train_x_adult = df_train_adult.iloc[:, :-1]
df_train_y_adult = df_train_adult.iloc[:, -1]
df_test_x_adult = df_test_adult.iloc[:, :-1]
df_test_y_adult = df_test_adult.iloc[:, -1]
df_test_sens_adult = df_test_adult['sex']

train_x_adult, test_x_adult = process_dfs(df_train_x_adult, df_test_x_adult, 
                                                   ['workclass', 'education','marital-status',
                                                    'occupation','relationship','race',
                                                    'native-country'])
train_y_adult = df_train_y_adult.values
test_y_adult = df_test_y_adult.values
test_sens_adult = df_test_sens_adult.values

# german
# the outcome (credit status) is the last column
df_train_x_german = df_train_german.iloc[:, :-1]
df_train_y_german = df_train_german.iloc[:, -1]
df_test_x_german = df_test_german.iloc[:, :-1]
df_test_y_german = df_test_german.iloc[:, -1]
df_test_sens_german = df_test_german['age']

train_x_german, test_x_german = process_dfs(df_train_x_german, df_test_x_german,
                                                     ['checking_account', 'credit_history', 
                                                      'purpose', 'savings_account', 'present_employment_since', 
                                                      'personal_status_sex', 'other_debtors',
                                                     'property', 'other_installment_plans',
                                                     'housing', 'job', 'telephone', 'foreign_worker'])
train_y_german = df_train_y_german.values
test_y_german = df_test_y_german.values
test_sens_german = df_test_sens_german.values

print(train_x_adult.shape, test_x_adult.shape, train_y_adult.shape, test_y_adult.shape)
print(train_x_german.shape, test_x_german.shape, train_y_german.shape, test_y_german.shape)

(30162, 103) (15060, 103) (30162,) (15060,)
(700, 61) (300, 61) (700,) (300,)


In [12]:
# train a classifier to predict the outcome y from features x
# training: train_x --> train_y; test: test_x --> preds
# logistic regression model is recommended
# sklearn is allowed to use
from sklearn.linear_model import LogisticRegression


# Adult

# initialize the model
# TODO. 3pts
model_adult = LogisticRegression(solver='liblinear')

# train/fit the model with train_x_adult and train_y_adult
# TODO. 4pts
model_adult.fit(train_x_adult, train_y_adult)

# predict the outcome from test_x_adult
# TODO. 3pts. The starter code below just indicate what you need to output in your code.
preds = model_adult.predict(test_x_adult)


# report acc and two fairness metrics. 
from sklearn.metrics import accuracy_score
acc = accuracy_score(test_y_adult, preds)
stat_p = stat_parity(preds, test_sens_adult)
eq_op = eq_oppo(preds, test_sens_adult, test_y_adult)
print(acc, stat_p, eq_op)





# German

# initialize the model
# TODO. 3pts
model_german = LogisticRegression(solver='liblinear')

# train/fit the model with train_x_german and train_y_german
# TODO. 4pts
model_german.fit(train_x_german, train_y_german)


# predict the outcome from test_x_german
# TODO. 3pts. The starter code below just indicate what you need to output in your code.
preds = model_german.predict(test_x_german)


# report acc and two fairness metrics
from sklearn.metrics import accuracy_score
acc = accuracy_score(test_y_german, preds)
stat_p = stat_parity(preds, test_sens_german)
eq_op = eq_oppo(preds, test_sens_german, test_y_german)
print(acc, stat_p, eq_op)

0.8460823373173971 0.184314312558775 0.18437865629412392
0.7566666666666667 -0.10337468320661602 -0.1276872861940182


## 4. Explore possible ways to mitigate bias

### 4. 1 remove protected attribute

In [13]:
# Adult
# remove the sex column from df_train_x_adult and df_test_x_adult. 
# You shouldn't do it in-place. In other words, do not modify df_train_x_adult or df_test_x_adult
# TODO. 2pts. The starter code below just indicate what you need to output in your code.
df_train_x_no_sens_adult = df_train_x_adult.drop('sex', axis=1)
df_test_x_no_sens_adult = df_test_x_adult.drop('sex', axis=1)


train_x_adult, test_x_adult = process_dfs(df_train_x_no_sens_adult, df_test_x_no_sens_adult, 
                                                   ['workclass', 'education','marital-status',
                                                    'occupation','relationship','race',
                                                    'native-country'])


# German
# remove age column from df_train_x_german and df_test_x_german
# You shouldn't do it in-place. In other words, do not modify df_train_x_german or df_test_x_german
# TODO. 2pts. The starter code below just indicate what you need to output in your code.
df_train_x_no_sens_german = df_train_x_german.drop('age', axis=1)
df_test_x_no_sens_german = df_test_x_german.drop('age', axis=1)


train_x_german, test_x_german = process_dfs(df_train_x_no_sens_german, df_test_x_no_sens_german,
                                                     ['checking_account', 'credit_history', 
                                                      'purpose', 'savings_account', 'present_employment_since', 
                                                      'personal_status_sex', 'other_debtors',
                                                     'property', 'other_installment_plans',
                                                     'housing', 'job', 'telephone', 'foreign_worker'])


print(train_x_adult.shape, test_x_adult.shape)
print(train_x_german.shape, test_x_german.shape)

(30162, 102) (15060, 102)
(700, 60) (300, 60)


In [14]:
# train a classifier to predict the outcome y from features x (with protected feature removed)
# training: train_x --> train_y; test: test_x --> preds
# logistic regression model is recommended
# sklearn is allowed to use
# Just use the code in 3.2 again


# Adult

# initialize the model
# TODO. 0pt
model_no_sens_adult = LogisticRegression(solver='liblinear')

# train/fit the model with train_x_adult and train_y_adult
# TODO. 0pt
model_no_sens_adult.fit(train_x_adult, train_y_adult)

# predict the outcome from test_x_adult
# TODO. 0pt. The starter code below just indicate what you need to output in your code.
preds = model_no_sens_adult.predict(test_x_adult)

# report acc and two fairness metrics
from sklearn.metrics import accuracy_score
acc = accuracy_score(test_y_adult, preds)
stat_p = stat_parity(preds, test_sens_adult)
eq_op = eq_oppo(preds, test_sens_adult, test_y_adult)
print(acc, stat_p, eq_op)



# German

# initialize the model
# TODO. 0pt
model_no_sens_german = LogisticRegression(solver='liblinear')

# train/fit the model with train_x_german and train_y_german
# TODO. 0pt
model_no_sens_german.fit(train_x_german, train_y_german)

# predict the outcome from test_x_german
# TODO. 0pt. The starter code below just indicate what you need to output in your code.
preds = model_no_sens_german.predict(test_x_german)

# report acc and two fairness metrics
from sklearn.metrics import accuracy_score
acc = accuracy_score(test_y_german, preds)
stat_p = stat_parity(preds, test_sens_german)
eq_op = eq_oppo(preds, test_sens_german, test_y_german)
print(acc, stat_p, eq_op)

0.8457503320053121 0.17471816846799434 0.1493626846346902
0.7633333333333333 -0.07643057222889149 -0.06114360700499011


### According to the results, how are the accuracy, stat parity and eq oppo different from the original model? Does explicitly removing the sensitive feature help in mitigating bias? Why or why not?
### <span style="color:red">Please type your response here.</span> 5pts

1. The **accuracy score** of predicting adult dataset drops for 0.033201% which does not make a big difference in current circumstances. The accuracy score of german dataset increased for 0.666667%, indicating removing the sensitive feature could potentially improve the prediction but largely still does not show enough edivence of dependence.

2. The **statistical parity** of adult dataset *drops* by 0.959614%, and that of german dataset *drops* by 2.694411%, both calculated in absolute values.

2. The **Equalized oppo** of adult dataset *drops* by -3.501597%, and that of german dataset *drops* by 6.654368%, both calculated in absolute values.

---

Explicitly removing the sensitive feature does not necessarily completely remove bias but it can take effect to some extend. Reasons:

1. Fairness Metrics decreased:

Statistical parity and equalized opportunity both decreased, meaning reduced group disparities and improved fairness when the sensitive attribute was removed.

2. Indirect Correlations Persist:

Even if we remove sex from the Adult dataset and age from the German dataset, other features in the dataset may still be correlated with these sensitive attributes. For example, in the Adult dataset, occupation, marital status, and relationship may serve as proxies for sex. In the German dataset, features like employment status or credit history might correlate with age.
As a result, the model can still infer the sensitive feature from other variables, leading to biased predictions despite its removal.

3. Accuracy Trade-off:

In the German dataset, accuracy slightly improved after removing the sensitive feature, but in the Adult dataset, it slightly dropped. This shows that sensitive features may sometimes contribute useful information for prediction, so their removal does not always improve fairness.

### 4.2 Augmenting the training set

#### See the example in Figure 1 of https://dl.acm.org/doi/pdf/10.1145/3375627.3375865

In [18]:
# Adult
# create a synthetic training set by duplicating df_train_x_adult and df_train_y_adult
# after duplicating flip sex in the synthetic set
# You shouldn't do it in-place. In other words, do not modify df_train_x_adult or df_train_y_adult
# TODO. 3pts. The starter code below just indicate what you need to output in your code.
df_train_x_syn_adult = df_train_x_adult.copy()
df_train_y_syn_adult = df_train_y_adult.copy()

df_train_x_syn_adult['sex'] = 1 - df_train_x_syn_adult['sex']

# augment the original training set by the synthetic set. In other words, concatenate them
df_train_x_aug_adult = pd.concat((df_train_x_adult, df_train_x_syn_adult))
df_train_y_aug_adult = pd.concat((df_train_y_adult, df_train_y_syn_adult))

print("df_train_x_aug_adult: ", df_train_x_aug_adult.shape, "\ndf_train_y_aug_adult: ", df_train_y_aug_adult.shape)


train_x_adult, test_x_adult = process_dfs(df_train_x_aug_adult, df_test_x_adult, 
                                                   ['workclass', 'education','marital-status',
                                                    'occupation','relationship','race',
                                                    'native-country'])
train_y_adult = df_train_y_aug_adult.values
print("\ntrain_x_adult: ", train_x_adult.shape, "\ntest_x_adult: ", test_x_adult.shape, "\ntrain_y_adult: ", train_y_adult.shape)



# German
# create a synthetic training set by duplicating df_train_x_german and df_train_y_german
# after duplicating flip age in the synthetic set.
# You shouldn't do it in-place. In other words, do not modify df_train_x_german or df_train_y_german
# TODO. 3pts. The starter code below just indicate what you need to output in your code.
df_train_x_syn_german = df_train_x_german.copy()
df_train_y_syn_german = df_train_y_german.copy()

df_train_x_syn_german['age'] = 1 - df_train_x_syn_german['age']

# augment the original training set by the synthetic set. In other words, concatenate them
df_train_x_aug_german = pd.concat((df_train_x_german, df_train_x_syn_german))
df_train_y_aug_german = pd.concat((df_train_y_german, df_train_y_syn_german))

train_y_german = df_train_y_aug_german.values

print(df_train_x_aug_german.shape, df_train_y_aug_german.shape, train_y_german.shape)


train_x_german, test_x_german = process_dfs(df_train_x_aug_german, df_test_x_german,
                                                     ['checking_account', 'credit_history', 
                                                      'purpose', 'savings_account', 'present_employment_since', 
                                                      'personal_status_sex', 'other_debtors',
                                                     'property', 'other_installment_plans',
                                                     'housing', 'job', 'telephone', 'foreign_worker'])
print(train_x_german.shape, test_x_german.shape)

df_train_x_aug_adult:  (60324, 14) 
df_train_y_aug_adult:  (60324,)

train_x_adult:  (60324, 103) 
test_x_adult:  (15060, 103) 
train_y_adult:  (60324,)
(1400, 20) (1400,) (1400,)
(1400, 61) (300, 61)


In [16]:
# train a classifier to predict the outcome y from features x on the augmented training data
# training: train_x --> train_y; test: test_x --> preds
# logistic regression model is recommended
# sklearn is allowed to use
# Just use the code in 3.2 again


# Adult

# initialize the model
# TODO. 3pts
model_adult = LogisticRegression(solver='liblinear')

# train/fit the model with train_x_adult and train_y_adult
# TODO. 4pts
model_adult.fit(train_x_adult, train_y_adult)

# predict the outcome from test_x_adult
# TODO. 3pts. The starter code below just indicate what you need to output in your code.
preds = model_adult.predict(test_x_adult)


# report acc and two fairness metrics. 
from sklearn.metrics import accuracy_score
acc = accuracy_score(test_y_adult, preds)
stat_p = stat_parity(preds, test_sens_adult)
eq_op = eq_oppo(preds, test_sens_adult, test_y_adult)
print(acc, stat_p, eq_op)



# German

# initialize the model
# TODO. 3pts
model_german = LogisticRegression(solver='liblinear')

# train/fit the model with train_x_german and train_y_german
# TODO. 4pts
model_german.fit(train_x_german, train_y_german)


# predict the outcome from test_x_german
# TODO. 3pts. The starter code below just indicate what you need to output in your code.
preds = model_german.predict(test_x_german)


# report acc and two fairness metrics
from sklearn.metrics import accuracy_score
acc = accuracy_score(test_y_german, preds)
stat_p = stat_parity(preds, test_sens_german)
eq_op = eq_oppo(preds, test_sens_german, test_y_german)
print(acc, stat_p, eq_op)

0.846879150066401 0.17517229075356358 0.14869682804369228
0.7633333333333333 -0.07643057222889149 -0.06114360700499011


### According to the results, how are the accuracy, stat parity and eq oppo different from the original model? Does augmenting the dataset with synthetic data help in mitigating bias? Why or why not?
### <span style="color:red">Please type your response here.</span> 5pts

1. Accuracy
There is a slight increase in accuracy for both datasets after augmenting with synthetic data.
The improvement is more noticeable for the German dataset (+0.00667) compared to the Adult dataset (+0.0008).
2. Statistical Parity
Adult Dataset: Stat parity decreased slightly (-0.0091), meaning the gap between groups increased slightly.
German Dataset: Stat parity increased (+0.0269), which means the gap between groups was reduced.
3. Equalized Opportunity
Adult Dataset: Eq oppo decreased (-0.0357), suggesting an increased disparity in the true positive rates across groups.
German Dataset: Eq oppo increased significantly (+0.0665), meaning the true positive rates between groups became more balanced.

---

Question: Does Augmenting the Dataset with Synthetic Data Help in Mitigating Bias?
* For the German dataset: Yes, it seems to have reduced bias, as statistical parity and equalized opportunity both improved while accuracy remained stable or increased.
* For the Adult dataset: No, it did not significantly mitigate bias. Although accuracy remained stable, both fairness metrics (stat parity and eq oppo) worsened slightly.

Reasons:
* Effectiveness depends on dataset structure: The German dataset benefited more from augmentation, possibly due to its smaller size or greater imbalance in the original data.
* Synthetic balancing does not always improve fairness: While balancing the dataset by flipping sex creates a more representative distribution, it does not necessarily correct underlying biases in the model's decision-making.
* Trade-off between accuracy and fairness: In the Adult dataset, fairness metrics worsened slightly, suggesting that bias might still exist even after augmentation.

**Conclusion:**

Augmenting the dataset with synthetic data may help in mitigating bias, but its effectiveness depends on the dataset. In this case, it helped for the German dataset but not for the Adult dataset.